In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
import numpy as np 
import matplotlib.pyplot as plt 

In [3]:
#Residual blocks 
#Id function allows us to train a bigger network with the same layers.
    #=> We put X, keep X + a residual block learning the changes on X
    #=> Weights ---> 0 
    

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

LABELS = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
HEIGHT = x_train.shape[1]
WIDTH = x_train.shape[2]
INPUT_SHAPE = (HEIGHT, WIDTH, 3)

(50000, 32, 32, 3)
(50000, 1)


In [7]:
#Let's build the model ! 

#Resnet Block ! 
def residual_block(x, kernel, nb_filters, reduce=False): 

    result = x
    if reduce: 
        #Reduce shape => stride = 2  + 1x1
        #First conv if reduce 
        conv_result = tf.keras.layers.Conv2D(
            strides=2, 
            kernel_size=1,
            filters=nb_filters,
            padding="SAME", 
            input_shape=x.shape[1::]
        )(result)
        #batch norm after each convolution 
        batch_norm_result = tf.keras.layers.BatchNormalization()(conv_result)
        result = tf.keras.layers.ReLU(batch_norm_result)
    #Anyways run the first imposed conv
    conv_result = tf.keras.layers.Conv2D(
        strides=1, 
        kernel_size=kernel,
        filters=nb_filters,
        padding="SAME", 
        input_shape=x.shape[1::]
    )(result)
    #Batch norm and activation (RELU)
    batch_norm_result = tf.keras.layers.BatchNormalization()(conv_result)
    result = tf.keras.layers.ReLU(batch_norm_result)
    #Second imposed convolution 
    conv_result2 = tf.keras.layers.Conv2D(
        strides=1, 
        kernel_size=kernel, 
        filters=nb_filters, 
        padding="SAME", 
        input_shape=x.shape[1::]
    )(result)
    #batch norm after each convolution 
    batch_norm_result2 = tf.keras.layers.BatchNormalization()(conv_result2)
    #F(x) + x
    block_result = batch_norm_result2 + x
    #Using relu on the addition
    activation_block_result = tf.keras.layers.ReLU(block_result)
    return activation_block_result     

In [ ]:
#TODO 
# filters = [] => idx i = block i 
# nbr_filters = []
# res_block_nbr = [3, 4, 6, 3]
def ResNet(x, filters, nbr_filters, res_block_nbr): 
    
    #start resnet HERE : 
        #7x7, 64 filters, stride = 2, padding = "same"
        #3x3 max pool, stride = 2
    
    #We assume we are in an 18 layer mindset => only 2 blocks of convolution by 
    result = x
    for step in range(len(filters)): 
        #Processing each convs
        #First Block = Reducing dimensions
        result = residual_block(
                    x=result, 
                    nb_filters=nbr_filters[step], 
                    kernel=filters[step], 
                    reduce=True
                )
        #Other blocks
        for i in range(1,nbr_block_nbr): 
            result = residual_block(
                        x=result, 
                        nb_filters=nbr_filters[step], 
                        kernel=filters[step], 
                        reduce=False
                    )
    #Average pooling 
    result_final_pool = tf.keras.layers.AveragePooling2D()(result)
    #returining probabilities
    final_fc = tf.keras.layers.Dense(units=1000, activation="softmax")(result_final_pool)
    return final_fc